In [1]:
import pandas as pd
import os

## 1. Download the data

In [2]:
data = 'viame_sp'
output_dir = "/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/"
output_path = f"{output_dir}{data}.zip"
download_link = "https://viame.kitware.com/api/v1/dive_dataset/export?folderIds=[%2265c39a4ea8df0635ce3b443e%22,%2265c6378785591d30c22fbe9e%22,%2265b7e3e1a4c218785d460e30%22,%2265c5a73263ff94bc31466fd9%22,%2265c5a779b3a7cddd61cf098d%22,%2265b90381a4c218785d462001%22,%2265baad76cf5a99794eaf0791%22,%2265baae0bd17f7c8fa5bf73b5%22,%2265baaef8a4c218785d463051%22,%2265c3a78a8cb22dc59f13f1af%22,%2265c3fb40ecb4b390ccc31d41%22,%2265c3fc7585591d30c22f64de%22,%2265c5a7148cb22dc59f144e17%22]"

In [3]:
!wget -O {output_path} {download_link}
!unzip {output_path} -d {output_dir}

--2024-03-09 10:18:10--  https://viame.kitware.com/api/v1/dive_dataset/export?folderIds=[%2265c39a4ea8df0635ce3b443e%22,%2265c6378785591d30c22fbe9e%22,%2265b7e3e1a4c218785d460e30%22,%2265c5a73263ff94bc31466fd9%22,%2265c5a779b3a7cddd61cf098d%22,%2265b90381a4c218785d462001%22,%2265baad76cf5a99794eaf0791%22,%2265baae0bd17f7c8fa5bf73b5%22,%2265baaef8a4c218785d463051%22,%2265c3a78a8cb22dc59f13f1af%22,%2265c3fb40ecb4b390ccc31d41%22,%2265c3fc7585591d30c22f64de%22,%2265c5a7148cb22dc59f144e17%22]
Resolving viame.kitware.com (viame.kitware.com)... 216.136.40.55
Connecting to viame.kitware.com (viame.kitware.com)|216.136.40.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/viame_sp.zip’

          /vol/biom     [<=>                 ]       0  --.-KB/s               

/vol/biomedic3/bglo     [               <=>  ]   3.39G  15.8MB/s    in 1m 58s  

2024-03-09 10:20:08 (29.4 MB/s) - ‘/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/viame_sp.zip’ saved [3635900963]

Archive:  /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/viame_sp.zip
 extracting: /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/natgeo/meta.json  
 extracting: /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/natgeo/annotations.dive.json  
 extracting: /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/natgeo/20220924_KeawanuiBay_S3_GX110006.mp4  
 extracting: /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/natgeo/20220924_KeawanuiBay_S3_GX110006.transcoded.aligned.mp4  
 extracting: /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/natgeo/annotations.viame.csv  
 extracting: /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev

In [8]:
for source in os.listdir(output_dir):
  if not any([source.startswith(prefix) for prefix in ['gfp', 'val1', 'shlife']]):
    print(source)

sp_palau6
sp_palau10.mp4
sp_palau5
sp_palau8
sp_palau2
sp_palau9.mp4
sp_palau
sp_palau7
sp_palau3
sp_palau11
sp_palau4
sp_natgeo2
sp_natgeo


## 2. Extract Frames
Some data is image sequence, so we already have the frames. Others are video sequences, so we need to extract them.


Therefore, have a ANNOTATIONS FIRST approach:
1. Extract the annotations
2. From the annotations select the frames you need

### 2.1 Convert annotations to sharktrack format

In [ ]:
SHARKTRACK_COLUMNS = ['filename', 'class', 'ymin', 'xmin', 'xmax', 'ymax', 'source', 'track_id', 'frame_id']

def viame2standard(csv_path, source, is_video, annotations_fps):
    assert annotations_fps in [1,10], "Only 1 or 10 fps are supported"
    # Load the CSV file
    df = pd.read_csv(csv_path, skiprows=lambda x: x in [1]) # skip row if metadata

    data = []

    for _, row in df.iterrows():
        # Build the Filename
        track_id = int(row["# 1: Detection or Track-id"])
        frame_id = int(row['3: Unique Frame Identifier'])

        if annotations_fps == 10 and frame_id % 10 != 0:
            # extract only 1fps annotations
            continue
    
        frame_id = int(frame_id / annotations_fps) # id starts at 0 and increments by 1 (if fps 1 doesn't change)

        if is_video:
            filename = f"frame{frame_id}.jpg"
        else:
            filename = row['2: Video or Image Identifier']
        img_path = os.path.join(output_dir, source, filename)
        assert os.path.exists(img_path), f"File {img_path} does not exist"

        species = row["10-11+: Repeated Species"]
        # TODO: filter out behaviour classification and clean only to shark/ray
        
        xmin = row['4-7: Img-bbox(TL_x']
        ymin = row['TL_y']
        xmax = row['BR_x']
        ymax = row['BR_y)']
        
        # Prepare the new row as a Series
        row = {
            'filename': f'{source}_{filename}',
            'class': species,
            'ymin': ymin,
            'xmin': xmin,
            'xmax': xmax,
            'ymax': ymax,
            'source': source,
            'track_id': track_id,
            'frame_id': frame_id,
        }

        data.append(row)

    converted_df = pd.DataFrame(data, columns=SHARKTRACK_COLUMNS)

    return converted_df

### 2.2 Move frames

In [ ]:
phase3_path = '/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3'

def process_image_sequence(image_sequence_path):
    source = os.path.basename(image_sequence_path)
    os.makedirs(os.path.join(phase3_path, source))

    # Convert annotations to sharktrack format
    #   annotations.csv(path)
    # Move frames there
    #   assert filename doesn't start with source and add it yourself
    #   rename frame to source_frame

In [ ]:
def process_video_annotations(video_path):
  # extract 1fps frames from video, name frame<id> where <id> starts at 0 and increments by 1
  # convert annotations to sharktrack format using same rule
  #   annotations.csv(path)
  # Move frames there
  #   assert filename doesn't start with source and add it yourself
  #   rename frame to source_frame

In [17]:
# If image: just move all the frames however they are named
# If video AND 1fps (sp), extract the frame at 1fps AND map it to annotation frame id
for video in os.listdir(output_dir):
  video_path = os.path.join(output_dir, video)
  if not os.path.isdir(video_path):
    continue

  for file in os.listdir(video_path):
    if file.endswith(".mp4"):
      print(f"{file} is video")
      break
    else:
      # Image Sequence!
      process_image_sequence(video_path)

easy2.transcoded.aligned.mp4 is video
medium2.mp4 is video
easy1.transcoded.aligned.mp4 is video
medium1.mp4 is video
difficult2.mp4 is video
difficult1.mp4 is video


## 3. Clean BBoxes

In [ ]:
# If image, just take the "2: Video or Image Identifier" column
# If 

## 4. Visualise Dataset and check everything is alright

## 5. Remove dumb FP